# Introduction to the Planner

The Planner is one of the fundamental concepts of the Semantic Kernel. It makes use of the collection of plugins that have been registered to the kernel and using AI, will formulate a plan to execute a given ask.

Read more about it [here](https://aka.ms/sk/concepts/planner).

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.0.1"
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, 1.0.1-preview"

#!import config/Settings.cs
#!import config/Utils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Kernel = Microsoft.SemanticKernel.Kernel;

var builder = Kernel.CreateBuilder();

// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

Installed Packages Microsoft.SemanticKernel, 1.0.1 Microsoft.SemanticKernel.Planners.Handlebars, 1.0.1-preview

### Setting Up Handlebars Planner
Handlebars Planner is located in the `Microsoft.SemanticKernel.Planning.Handlebars` package.

In [2]:
using Microsoft.SemanticKernel.Planning.Handlebars;

#pragma warning disable SKEXP0060

var planner = new HandlebarsPlanner();

### Providing plugins to the planner
The planner needs to know what plugins are available to it. Here we'll import the `SummarizePlugin` and `WriterPlugin` we have defined on disk.

In [3]:
var pluginsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "..", "..", "prompt_template_samples");

kernel.ImportPluginFromPromptDirectory(Path.Combine(pluginsDirectory, "SummarizePlugin"));
kernel.ImportPluginFromPromptDirectory(Path.Combine(pluginsDirectory, "WriterPlugin"));

Define your ASK. What do you want the Kernel to do?

In [4]:
#pragma warning disable SKEXP0060

var ask = "Tomorrow is Valentine's day. I need to come up with a few date ideas. My significant other likes poems so write them in the form of a poem.";
var originalPlan = await planner.CreatePlanAsync(kernel, ask);

Console.WriteLine("Original plan:\n");
Console.WriteLine(originalPlan);

Original plan:

{{!-- Step 1: Define the goal for the date Brainstorming --}}
{{set "dateGoal" "Valentine's day date ideas"}}

{{!-- Step 2: Generate date ideas using Brainstorm helper --}}
{{set "dateIdeas" (WriterPlugin-Brainstorm input=dateGoal)}}

{{!-- Step 3: Loop through each date idea and create a poem for each --}}
{{#each dateIdeas}}
    {{set "poemIdea" this}}
    {{set "generatedPoem" (WriterPlugin-ShortPoem input=poemIdea)}}
    {{!-- Step 4: Print each poem --}}
    {{json generatedPoem}}
{{/each}}


As you can see in the above plan, the Planner has taken the user's ask and converted it into a Plan object detailing how the AI would go about solving this task.

It makes use of the plugins that the Kernel has available to it and determines which functions to call in order to fulfill the user's ask.

Let's also define an inline plugin and have it be available to the Planner.
Be sure to give it a function name and plugin name.

In [5]:
string skPrompt = """
{{$input}}

Rewrite the above in the style of Shakespeare.
""";

var executionSettings = new OpenAIPromptExecutionSettings 
{
    MaxTokens = 2000,
    Temperature = 0.7,
    TopP = 0.5
};

var shakespeareFunction = kernel.CreateFunctionFromPrompt(skPrompt, executionSettings, "Shakespeare");

Let's update our ask using this new plugin.

In [6]:
#pragma warning disable SKEXP0060

var ask = @"Tomorrow is Valentine's day. I need to come up with a few date ideas.
She likes Shakespeare so write using his style. Write them in the form of a poem.";

var newPlan = await planner.CreatePlanAsync(kernel, ask);

Console.WriteLine("Updated plan:\n");
Console.WriteLine(newPlan);

Updated plan:

{{!-- Step 1: Identifying the notable item from the goal --}}
{{set "interest" "Shakespeare"}}

{{!-- Step 2: Generate date ideas considering her interest --}}
{{set "dateIdeas" (WriterPlugin-Brainstorm input="A Valentine's day date for someone who likes Shakespeare")}}

{{!-- Step 3: Write generated date ideas in the form of a poem in Shakespearean style --}}
{{set "datePoem" (WriterPlugin-ShortPoem input=dateIdeas)}}
{{set "datePoemShakespeareanStyle" (WriterPlugin-Rewrite style="Shakespearean" input=datePoem)}}

{{!-- Step 4: Output the final result --}}
{{json datePoemShakespeareanStyle}}


### Executing the plans

Now that we have different plans, let's try to execute them! The Kernel can execute the plan using RunAsync.

In [8]:
#pragma warning disable SKEXP0060

var originalPlanResult = await originalPlan.InvokeAsync(kernel, new KernelArguments());

Console.WriteLine("Original Plan results:\n");
Console.WriteLine(Utils.WordWrap(originalPlanResult.ToString(), 100));

Original Plan results:

{}



Now lets execute and print the new plan:

In [9]:
#pragma warning disable SKEXP0060

var newPlanResult = await newPlan.InvokeAsync(kernel, new KernelArguments());

Console.WriteLine("New Plan results:\n");
Console.WriteLine(newPlanResult);

New Plan results:

love, so pure and true,
   In the quiet of the night, under the moon's soft light,
   We shared a moment, a connection anew.

1. Unto a play of Shakespeare, we didst venture,
   A narrative of love or a tragic misadventure,
   With words so noble, in a local ensemble,
   We laughed and wept, oh what a spectacle!

2. A sonnet recital, solely for us two,
   With words of affection, so sincere and true,
   In the tranquility of the night, 'neath the moon's gentle light,
   We shared a moment, a bond that felt new.
